In [ ]:
from fastai.vision.all import *

import pandas as pd

In [ ]:
data_path = Path('/home/tanul/torchenv/data/tiles') #todo: move this out to /4TB
data_path.ls()

In [ ]:
LABELS_FILE = '/home/gosia/text-detection-internships/FasterRCNN Pipeline/train.csv'
labels_df = pd.read_csv(LABELS_FILE)
labels_df

In [ ]:
labels_df.label.unique()

In [ ]:
labels_df.loc[labels_df['label'] == ']']

In [ ]:
# Filter out 'background' annotations
#labels_df = labels_df[(labels_df['label']!='background')]

In [ ]:
def df_to_dict(df):
    out = {}
    for idx, row in df.iterrows():
        fname = row['image_id']
        annot = out.get(fname, ([], []))
        x = row['xmin']
        y = row['ymin']
        w = row['xmax'] - x
        h = row['ymax'] - y
        annot[0].append([x, y, x+w, y+h])
        lbl = row['label']
        if lbl not in ['text', 'digit', 'letter']:
            lbl = 'background'
        annot[1].append(lbl)
        out[fname] = annot
    return out

In [ ]:
img2bbox = df_to_dict(labels_df)
img2bbox

In [ ]:
get_y = [lambda o: img2bbox.get(o.name, ([],[]))[0],
         lambda o: img2bbox.get(o.name, ([],[]))[1]]
#item_tfms = [Resize(128, method='pad'),]
item_tfms = []
#batch_tfms = [Rotate(), Flip(), Dihedral(), Normalize.from_stats(*imagenet_stats)]
batch_tfms = []
dblocks = DataBlock(blocks=(ImageBlock, BBoxBlock, BBoxLblBlock),
                    splitter=RandomSplitter(),
                    get_items=get_image_files,
                    get_y=get_y,
                    item_tfms=item_tfms,
                    batch_tfms=batch_tfms,
                    n_inp=1
                   )

In [ ]:
dls = dblocks.dataloaders(data_path)

In [ ]:
#dblocks.summary(data_path)

In [ ]:
# from PIL import Image, ImageDraw
# img = dls.train_ds[3][0]
# draw = ImageDraw.Draw(img)

# bbxs = dls.train_ds[3][1]
# for bb in bbxs:
#     draw.rectangle(((bb[0], bb[1]), (bb[0]+bb[2], bb[1]+bb[3])), outline='red')
# img

In [ ]:
dls.show_batch(figsize=(20,20))